<a href="https://colab.research.google.com/github/BozIAymane/Big_data/blob/main/AlgoDis_SPARK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installation de PySpark**

In [1]:
!pip install pyspark

**Imports**

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import re

**Initialisation de Spark**

In [3]:
try:
    sc.stop()
except:
    pass

# Créer un nouveau contexte Spark
sc = SparkContext(appName='AndroidLogAnalysis')
print("SparkContext initialisé avec succès!")

SparkContext initialisé avec succès!


**Charger le fichier de logs**

In [4]:
fichier_log = "/content/Android.log"

log_RDD = sc.textFile(fichier_log)
print(f"✓ Fichier chargé avec succès!")
print(f"Nombre total de lignes: {log_RDD.count()}")

# Afficher les 3 premières lignes
print("\nAperçu des données:")
for ligne in log_RDD.take(3):
    print(ligne[:100] + "...")

✓ Fichier chargé avec succès!
Nombre total de lignes: 1555005

Aperçu des données:
12-17 19:31:36.263  1795  1825 I PowerManager_screenOn: DisplayPowerStatesetColorFadeLevel: level=1....
12-17 19:31:36.263  5224  5283 I SendBroadcastPermission: action:android.com.huawei.bone.NOTIFY_SPOR...
12-17 19:31:36.264  1795  1825 D DisplayPowerController: Animating brightness: target=21, rate=40...


**Fonction de parsing des logs Android**

In [5]:
def parse_android_log(line):
    """
    Parse une ligne de log Android
    Format: MM-DD HH:MM:SS.mmm PID TID LEVEL TAG: message
    """
    try:
        # Expression régulière pour le format Android
        pattern = r'(\d{2}-\d{2})\s+(\d{2}:\d{2}):(\d{2}\.\d{3})\s+(\d+)\s+(\d+)\s+([IWDEV])\s+([^:]+):'
        match = re.match(pattern, line)

        if match:
            date = match.group(1)        # MM-DD
            hour_min = match.group(2)    # HH:MM
            seconds = match.group(3)     # SS.mmm
            pid = match.group(4)         # Process ID
            tid = match.group(5)         # Thread ID
            level = match.group(6)       # Log level (I/W/D/E/V)
            tag = match.group(7).strip() # Tag

            return {
                'minute': hour_min,
                'level': level,
                'pid': pid,
                'tag': tag
            }
        return None
    except:
        return None

# Tester le parsing
sample = log_RDD.first()
parsed = parse_android_log(sample)
print("\n✓ Test de parsing:")
print(f"Ligne: {sample[:80]}...")
print(f"Résultat: {parsed}")


✓ Test de parsing:
Ligne: 12-17 19:31:36.263  1795  1825 I PowerManager_screenOn: DisplayPowerStatesetColo...
Résultat: {'minute': '19:31', 'level': 'I', 'pid': '1795', 'tag': 'PowerManager_screenOn'}


**Parser tous les logs**

In [6]:
parsed_logs_RDD = log_RDD.map(parse_android_log).filter(lambda x: x is not None)
nb_parsed = parsed_logs_RDD.count()
print(f"✓ Lignes parsées avec succès: {nb_parsed}")

✓ Lignes parsées avec succès: 1554809


**ANALYSE 1 - Nombre d'actions par minute**

In [7]:
print("\n" + "="*60)
print("📊 ANALYSE 1: Nombre d'actions par minute")
print("="*60)

actions_par_minute = (parsed_logs_RDD
                      .map(lambda x: (x['minute'], 1))
                      .reduceByKey(lambda a, b: a + b)
                      .sortByKey())

print("\nRésultats:")
resultats = actions_par_minute.collect()
for minute, count in resultats:
    print(f"  Minute {minute}: {count} actions")

print(f"\n✓ Total: {len(resultats)} minutes analysées")


📊 ANALYSE 1: Nombre d'actions par minute

Résultats:
  Minute 00:00: 2997 actions
  Minute 00:01: 808 actions
  Minute 00:02: 1352 actions
  Minute 00:03: 1299 actions
  Minute 00:04: 2064 actions
  Minute 00:05: 3021 actions
  Minute 00:06: 1182 actions
  Minute 00:07: 1242 actions
  Minute 00:08: 1136 actions
  Minute 00:09: 1786 actions
  Minute 00:10: 1679 actions
  Minute 00:11: 1221 actions
  Minute 00:12: 1823 actions
  Minute 00:13: 1937 actions
  Minute 00:14: 1665 actions
  Minute 00:15: 2211 actions
  Minute 00:16: 1976 actions
  Minute 00:17: 1780 actions
  Minute 00:18: 1895 actions
  Minute 00:19: 1410 actions
  Minute 00:20: 1891 actions
  Minute 00:21: 1925 actions
  Minute 00:22: 1600 actions
  Minute 00:23: 1645 actions
  Minute 00:24: 4730 actions
  Minute 00:32: 796 actions
  Minute 00:34: 172 actions
  Minute 00:35: 1 actions
  Minute 00:41: 97 actions
  Minute 00:54: 426 actions
  Minute 00:56: 469 actions
  Minute 01:04: 501 actions
  Minute 01:09: 20 actions
  

**ANALYSE 2 - Distribution par niveau de log**

In [8]:
print("\n" + "="*60)
print("📊 ANALYSE 2: Distribution par niveau de log")
print("="*60)

actions_par_niveau = (parsed_logs_RDD
                      .map(lambda x: (x['level'], 1))
                      .reduceByKey(lambda a, b: a + b)
                      .sortByKey())

level_names = {
    'I': 'INFO',
    'D': 'DEBUG',
    'W': 'WARNING',
    'E': 'ERROR',
    'V': 'VERBOSE'
}

print("\nRésultats:")
total = 0
for level, count in actions_par_niveau.collect():
    level_name = level_names.get(level, level)
    percentage = (count / nb_parsed) * 100
    print(f"  {level_name} ({level}): {count} actions ({percentage:.1f}%)")
    total += count


📊 ANALYSE 2: Distribution par niveau de log

Résultats:
  DEBUG (D): 505608 actions (32.5%)
  ERROR (E): 157829 actions (10.2%)
  INFO (I): 766082 actions (49.3%)
  VERBOSE (V): 44408 actions (2.9%)
  WARNING (W): 80882 actions (5.2%)


**ANALYSE 3 - Top 10 des tags les plus actifs**

In [9]:
print("\n" + "="*60)
print("📊 ANALYSE 3: Top 10 des tags les plus actifs")
print("="*60)

top_tags = (parsed_logs_RDD
            .map(lambda x: (x['tag'], 1))
            .reduceByKey(lambda a, b: a + b)
            .sortBy(lambda x: x[1], ascending=False)
            .take(10))

print("\nRésultats:")
for i, (tag, count) in enumerate(top_tags, 1):
    print(f"  {i:2d}. {tag:40s} : {count:5d} occurrences")



📊 ANALYSE 3: Top 10 des tags les plus actifs

Résultats:
   1. SDK                                      : 133646 occurrences
   2. PowerManagerService                      : 75836 occurrences
   3. HwCustMobileSignalControllerImpl         : 55644 occurrences
   4. wpa_supplicant                           : 53279 occurrences
   5. HwSignalClusterView                      : 38672 occurrences
   6. amsnl                                    : 30409 occurrences
   7. libteec                                  : 29722 occurrences
   8. UsbDeviceManager                         : 28579 occurrences
   9. SendBroadcastPermission                  : 24563 occurrences
  10. ActivityManager                          : 23781 occurrences


**ANALYSE 4 - Top processus (PID) les plus actifs**

In [10]:
print("\n" + "="*60)
print("📊 ANALYSE 4: Top 10 des processus (PID) les plus actifs")
print("="*60)

top_pids = (parsed_logs_RDD
            .map(lambda x: (x['pid'], 1))
            .reduceByKey(lambda a, b: a + b)
            .sortBy(lambda x: x[1], ascending=False)
            .take(10))

print("\nRésultats:")
for i, (pid, count) in enumerate(top_pids, 1):
    print(f"  {i:2d}. PID {pid:6s} : {count:5d} actions")


📊 ANALYSE 4: Top 10 des processus (PID) les plus actifs

Résultats:
   1. PID 1795   : 488930 actions
   2. PID 2852   : 352089 actions
   3. PID 633    : 155938 actions
   4. PID 5224   : 42253 actions
   5. PID 5157   : 36871 actions
   6. PID 3218   : 36738 actions
   7. PID 3626   : 35235 actions
   8. PID 3858   : 30836 actions
   9. PID 24266  : 26831 actions
  10. PID 1046   : 24346 actions


**ANALYSE 5 - Détails par minute et niveau**

In [11]:
print("\n" + "="*60)
print("📊 ANALYSE 5: Distribution détaillée (minute × niveau)")
print("="*60)

minute_level_dist = (parsed_logs_RDD
                     .map(lambda x: ((x['minute'], x['level']), 1))
                     .reduceByKey(lambda a, b: a + b)
                     .sortByKey())

print("\nRésultats:")
current_minute = None
for (minute, level), count in minute_level_dist.collect():
    if minute != current_minute:
        if current_minute is not None:
            print()  # Ligne vide entre les minutes
        current_minute = minute
        print(f"Minute {minute}:")

    level_name = level_names.get(level, level)
    print(f"  - {level_name:8s}: {count:3d} actions")

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  - INFO    :  58 actions
  - VERBOSE :   1 actions
  - WARNING :   7 actions

Minute 00:54:
  - DEBUG   :  65 actions
  - ERROR   : 172 actions
  - INFO    : 162 actions
  - VERBOSE :   8 actions
  - WARNING :  19 actions

Minute 00:56:
  - DEBUG   : 160 actions
  - ERROR   :  48 actions
  - INFO    : 239 actions
  - VERBOSE :   2 actions
  - WARNING :  20 actions

Minute 01:04:
  - DEBUG   :  99 actions
  - ERROR   : 172 actions
  - INFO    : 214 actions
  - VERBOSE :   1 actions
  - WARNING :  15 actions

Minute 01:09:
  - DEBUG   :   2 actions
  - ERROR   :  17 actions
  - INFO    :   1 actions

Minute 01:19:
  - DEBUG   :   2 actions
  - ERROR   :  17 actions
  - INFO    :   1 actions

Minute 01:22:
  - DEBUG   :  43 actions
  - ERROR   :  26 actions
  - INFO    : 106 actions
  - VERBOSE :   3 actions
  - WARNING :  10 actions

Minute 01:26:
  - DEBUG   : 187 actions
  - ERROR   : 196 actions
  - INFO   

In [17]:
print("\n" + "="*60)
print("📊 ANALYSE 6: Nombre d'actions par heure")
print("="*60)

# Modifier la fonction de parsing pour extraire l'heure
def parse_android_log_with_hour(line):
    try:
        pattern = r'(\d{2}-\d{2})\s+(\d{2}):(\d{2}):(\d{2}\.\d{3})\s+(\d+)\s+(\d+)\s+([IWDEV])\s+([^:]+):'
        match = re.match(pattern, line)

        if match:
            # MM-DD HH:MM:SS.mmm PID TID LEVEL TAG: message
            hour = match.group(2) # HH

            # Réutiliser le parsing existant pour les autres champs si nécessaire, ou créer un nouveau dict
            return {
                'hour': hour,
                'level': match.group(7), # Log level
                'pid': match.group(5), # Process ID
                'tag': match.group(8).strip() # Tag
            }
        return None
    except:
        return None

# Appliquer la nouvelle fonction de parsing pour obtenir les heures
parsed_logs_with_hour_RDD = log_RDD.map(parse_android_log_with_hour).filter(lambda x: x is not None)

actions_par_heure = (parsed_logs_with_hour_RDD
                     .map(lambda x: (x['hour'], 1))
                     .reduceByKey(lambda a, b: a + b)
                     .sortByKey())

print("\nRésultats:")
resultats_heure = actions_par_heure.collect()
for hour, count in resultats_heure:
    print(f"  Heure {hour}: {count} actions")

print(f"\n✓ Total: {len(resultats_heure)} heures analysées")


📊 ANALYSE 6: Nombre d'actions par heure

Résultats:
  Heure 00: 48236 actions
  Heure 01: 3998 actions
  Heure 02: 2436 actions
  Heure 03: 2400 actions
  Heure 04: 2243 actions
  Heure 05: 2010 actions
  Heure 06: 2366 actions
  Heure 07: 68622 actions
  Heure 08: 96863 actions
  Heure 09: 58555 actions
  Heure 10: 63337 actions
  Heure 11: 17529 actions
  Heure 12: 23303 actions
  Heure 13: 7824 actions
  Heure 14: 144006 actions
  Heure 15: 120131 actions
  Heure 16: 14270 actions
  Heure 17: 77150 actions
  Heure 18: 64810 actions
  Heure 19: 167151 actions
  Heure 20: 138489 actions
  Heure 21: 198109 actions
  Heure 22: 96638 actions
  Heure 23: 134333 actions

✓ Total: 24 heures analysées


**STATISTIQUES GLOBALES**

In [12]:
print("\n" + "="*60)
print("📊 STATISTIQUES GLOBALES")
print("="*60)

total_logs = parsed_logs_RDD.count()
unique_minutes = parsed_logs_RDD.map(lambda x: x['minute']).distinct().count()
unique_pids = parsed_logs_RDD.map(lambda x: x['pid']).distinct().count()
unique_tags = parsed_logs_RDD.map(lambda x: x['tag']).distinct().count()

print(f"\n📈 Nombre total de logs           : {total_logs:,}")
print(f"⏰ Nombre de minutes différentes  : {unique_minutes}")
print(f"🔧 Nombre de processus (PID)      : {unique_pids}")
print(f"🏷️  Nombre de tags uniques         : {unique_tags}")

if unique_minutes > 0:
    avg_per_minute = total_logs / unique_minutes
    print(f"📊 Moyenne d'actions par minute   : {avg_per_minute:.2f}")



📊 STATISTIQUES GLOBALES

📈 Nombre total de logs           : 1,554,809
⏰ Nombre de minutes différentes  : 765
🔧 Nombre de processus (PID)      : 642
🏷️  Nombre de tags uniques         : 1741
📊 Moyenne d'actions par minute   : 2032.43


**Sauvegarder les résultats**

In [13]:
print("\n" + "="*60)
print("💾 Sauvegarde des résultats")
print("="*60)

# Sauvegarder les actions par minute
try:
    actions_par_minute.saveAsTextFile("output_actions_par_minute")
    print("✓ Actions par minute → output_actions_par_minute/")
except:
    print("⚠️  Dossier déjà existant, supprimez-le d'abord si besoin")

# Sauvegarder la distribution par niveau
try:
    actions_par_niveau.saveAsTextFile("output_actions_par_niveau")
    print("✓ Actions par niveau → output_actions_par_niveau/")
except:
    print("⚠️  Dossier déjà existant")

print("\n✅ Analyse terminée avec succès!")


💾 Sauvegarde des résultats
✓ Actions par minute → output_actions_par_minute/
✓ Actions par niveau → output_actions_par_niveau/

✅ Analyse terminée avec succès!


**Afficher le contenu des fichiers sauvegardés**

In [14]:
print("\n" + "="*60)
print("📂 Aperçu des fichiers sauvegardés")
print("="*60)

# Lire le fichier de sortie
import os
if os.path.exists("output_actions_par_minute"):
    print("\nContenu de output_actions_par_minute/part-00000:")
    with open("output_actions_par_minute/part-00000", "r") as f:
        print(f.read())


📂 Aperçu des fichiers sauvegardés

Contenu de output_actions_par_minute/part-00000:
('00:00', 2997)
('00:01', 808)
('00:02', 1352)
('00:03', 1299)
('00:04', 2064)
('00:05', 3021)
('00:06', 1182)
('00:07', 1242)
('00:08', 1136)
('00:09', 1786)
('00:10', 1679)
('00:11', 1221)
('00:12', 1823)
('00:13', 1937)
('00:14', 1665)
('00:15', 2211)
('00:16', 1976)
('00:17', 1780)
('00:18', 1895)
('00:19', 1410)
('00:20', 1891)
('00:21', 1925)
('00:22', 1600)
('00:23', 1645)
('00:24', 4730)
('00:32', 796)
('00:34', 172)
('00:35', 1)
('00:41', 97)
('00:54', 426)
('00:56', 469)
('01:04', 501)
('01:09', 20)
('01:19', 20)
('01:22', 188)
('01:26', 670)
('01:29', 19)
('01:35', 37)
('01:36', 27)
('01:37', 1998)
('01:56', 518)
('02:05', 39)
('02:06', 27)
('02:23', 746)
('02:24', 39)
('02:26', 501)
('02:35', 503)
('02:39', 28)
('02:49', 29)
('02:56', 496)
('02:59', 28)
('03:05', 27)
('03:25', 864)
('03:26', 698)
('03:35', 231)
('03:36', 48)
('03:56', 532)
('04:01', 701)
('04:05', 111)
('04:06', 27)
('04:09

**fermer Spark**

In [ ]:
sc.stop()
print("SparkContext arrêté")